# generate new antonym pairs for specific topic

In [54]:
import gensim
from gensim.models import Word2Vec
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.models.word2vec import LineSentence

#loading trained model
model_norm = KeyedVectors.load_word2vec_format("./Dataset/word2vec_norm", binary=True)

### loading original antonyms

In [ ]:
list_antonym = []

with open('./Dataset/Antonym_sets/LenciBenotto.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))


with open('./Dataset/Antonym_sets/LenciBenotto.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
with open('./Dataset/Antonym_sets/EVALution.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
with open('./Dataset/Antonym_sets/EVALution.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
                
list_antonym = list(dict.fromkeys(list_antonym).keys())

In [ ]:
similarity_matrix = defaultdict(list)
for each_pair in tqdm(list_antonym):
    word1 = each_pair[0]
    word2 = each_pair[1]
    if word1 < word2:
        similarity_matrix[word1].append(word2)
    else:
        similarity_matrix[word2].append(word1)
    
all_similarity = defaultdict(dict)
for each_key in tqdm(similarity_matrix):
    for each_value in similarity_matrix[each_key]:
#         cosine_similarity([current_model[each_key]]
        all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])
    
final_antonym_list = []
for index_counter, each_key in enumerate(tqdm(all_similarity)):
#     print(each_key,all_similarity[each_key])
    listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
#     print(listofTuples)
    final_antonym_list.append((each_key, listofTuples[0][0]))
print(len(final_antonym_list))

list_antonym = final_antonym_list

In [ ]:
# store original antonyms
with open('./all_antonyms', 'w') as f:
    f.writelines([i+' ' +j+'\n' for i,j in list_antonym])

### use original antonyms to find the most similar word pairs related to specific topic

In [121]:
from collections import defaultdict
antonyms = {}
with open('./all_antonyms', 'r') as f:
    for i, line in enumerate(f):
        word_pair = line.split()
        antonyms[i] = word_pair

def select_antonym(topic, antonyms, n, model):
    similarity_score = defaultdict(float)
    for i, words in antonyms.items():
        # similarity_score[i] = max(model.similarity(topic,words[0]),model.similarity(topic,words[0]))
        similarity_score[i] = model.similarity(topic,words[0]) + model.similarity(topic,words[0])
    selected_antonyms = []
    sorted_similarity_score = sorted(similarity_score.items(), key = lambda x:x[1], reverse=True)
    for item in sorted_similarity_score[:n]:
        selected_antonyms.append(antonyms[item[0]])
    return selected_antonyms

In [122]:
# generate antonyms for topic 'country'
select_country = select_antonym('country', antonyms, 300, model_norm)
select_china = select_antonym('china', antonyms, 300, model_norm)
select_usa = select_antonym('germany', antonyms, 200, model_norm)

re = []
for i in select_country:
    if i in select_china:
        re.append(i)

#store selectd antonyms
# with open('./generated_words/CountryWordPairsByHand','w') as f:
#     f.writelines(i+' '+j +'\n' for i,j in re)

# for i in select_country:
#     if i in select_usa and i not in re:
#         print(i[0],i[1])

### use selected antonyms as root to generate new antonyms by Wordnet and API

In [169]:
#load root antonyms
with open('./generated_antonyms/CountryWordPairsByHand','r') as f:
    root = [line.strip('\n').split(' ') for line in f.readlines()]

In [119]:
# test case cell
#########################
# from nltk.corpus import wordnet as wn
# wordsets = wn.synsets('good')
# for word in wordsets:
#     if word.hyponyms():
#         hypon = word.hyponyms()
#         for h in hypon:
#             for w in h.lemmas():
#                 if w.antonyms():
#                     print(w.name(), w.antonyms()[0].name())
# re = [sets.name().split('.')[0] for sets in wordsets]
# print(sets.lemmas()[0].antonyms())

In [163]:
# generate antonyms from wordnet

from nltk.corpus import wordnet as wn

def generate_antonyms_wordnet(root):
    res = []
    for antonyms in root:
        #generate antonym pair from synonyms
        wordsets = wn.synsets(antonyms[0])
        for sets in wordsets:
            for w in sets.lemmas():
                word1 = w.name()
                if w.antonyms():
                    word2 = w.antonyms()[0].name()
                    word_pair = [word1, word2]
                    if word_pair not in res:
                        res.append(word_pair)
        wordsets = wn.synsets(antonyms[1])
        for sets in wordsets:
            for w in sets.lemmas():
                word1 = w.name()
                if w.antonyms():
                    word2 = w.antonyms()[0].name()
                    word_pair = [word1, word2]
                    if word_pair not in res:
                        res.append(word_pair)
        #generate antonym pair from hyponyms
        wordsets = wn.synsets(antonyms[0])
        for sets in wordsets:
            if sets.hyponyms():
                hypon = sets.hyponyms()
                for h in hypon:
                    for w in h.lemmas():
                        word1 = w.name()
                        if w.antonyms():
                            word2 = w.antonyms()[0].name()
                            word_pair = [word1, word2]
                            if word_pair not in res:
                                 res.append(word_pair)
        wordsets = wn.synsets(antonyms[1])
        for sets in wordsets:
            if sets.hyponyms():
                hypon = sets.hyponyms()
                for h in hypon:
                    for w in h.lemmas():
                        word1 = w.name()
                        if w.antonyms():
                            word2 = w.antonyms()[0].name()
                            word_pair = [word1, word2]
                            if word_pair not in res:
                                res.append(word_pair)

    return res

In [164]:
antonyms_wordnet = generate_antonyms_wordnet(root)

In [112]:
#generate antonyms from API: http://www.thesaurus.com
import requests
from bs4 import BeautifulSoup

def find_synonyms(term):
    response = requests.get('http://www.thesaurus.com/browse/{}'.format(term))
    soup = BeautifulSoup(response.text, 'html.parser')
    section = soup.find('ul', {'class': 'css-17d6qyx-WordGridLayoutBox et6tpn80'})
    if section:
        # find 10 synonyms
        return [li.text for li in section.findAll('li')][:10]
    else:
        return

def find_antonyms(term):
    response = requests.get('http://www.thesaurus.com/browse/{}'.format(term))
    soup = BeautifulSoup(response.text, 'html.parser')
    soup('ul', {'class': 'css-17d6qyx-WordGridLayoutBox et6tpn80'})[0].extract()
    section = soup.find('ul', {'class': 'css-17d6qyx-WordGridLayoutBox et6tpn80'})
    if section:
        # find 3 antonyms
        return [li.text for li in section.findAll('li')][:3]
    else:
        return 

def generate_antonyms_api(root):
    res = []
    for antonyms in root:
        sy_1 = find_synonyms(antonyms[0])
        sy_2 = find_synonyms(antonyms[1])
        for s in sy_1:
            ant = find_antonyms(s)
            if ant:
                for a in ant:
                    word_pair = [s, a]
                    res.append(word_pair)
        for s in sy_2:
            ant = find_antonyms(s)
            if ant:
                for a in ant:
                    word_pair = [s, a]
                    res.append(word_pair)
    return res


In [139]:
%%time
antonyms_api = generate_antonyms_api(root)

Wall time: 7min 55s


In [157]:
# deal with generated antonym pairs
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics.pairwise import cosine_similarity
# first method: naive (for generated antonyms from wordnet)
def process_antonyms(antonyms, trained_model):
    # just reserve the unique word pairs which are contained in trained_model
    hash_set = set()
    for wordpair in antonyms:
        word1, word2 = wordpair
        if word1 in trained_model and word2 in trained_model:
            if word1 > word2:
                word1, word2 = word2, word1
            hash_set.add((word1, word2))
    print(len(hash_set))    
    return list(hash_set)

#second method: more powerful (for generated antonyms from api)
def process_antonyms_max(antonyms, current_model):
    hash_set = set()
    for wordpair in antonyms:
        word1, word2 = wordpair
        if word1 in current_model and word2 in current_model:
            if word1 > word2:
                word1, word2 = word2, word1
            hash_set.add((word1, word2))
    list_antonym = list(hash_set)

    similarity_matrix = defaultdict(list)
    for each_pair in tqdm(list_antonym):
        word1 = each_pair[0]
        word2 = each_pair[1]
        if word1 < word2:
            similarity_matrix[word1].append(word2)
        else:
            similarity_matrix[word2].append(word1)

    all_similarity = defaultdict(dict)
    for each_key in tqdm(similarity_matrix):
        for each_value in similarity_matrix[each_key]:
            all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])

    final_antonym_list = []
    for index_counter, each_key in enumerate(tqdm(all_similarity)):
        listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
        final_antonym_list.append((each_key, listofTuples[0][0]))

    print(len(final_antonym_list))
    return final_antonym_list

In [165]:
processed_antonyms_wordnet = process_antonyms(antonyms_wordnet, model_norm)
processed_antonyms_api = process_antonyms_max(antonyms_api, model_norm)

146



1023


In [166]:
def collect_antonyms(root, processed_antonyms_wordnet, processed_antonyms_api, current_model):
    hash_set = set()
    for wordpair in root:
        word1, word2 = wordpair
        hash_set.add((word1, word2))
    for wordpair in processed_antonyms_wordnet:
        word1, word2 = wordpair
        hash_set.add((word1, word2))
    for wordpair in processed_antonyms_api:
        word1, word2 = wordpair
        hash_set.add((word1, word2))

    list_antonym = list(hash_set)

    similarity_matrix = defaultdict(list)
    for each_pair in list_antonym:
        word1 = each_pair[0]
        word2 = each_pair[1]
        if word1 < word2:
            similarity_matrix[word1].append(word2)
        else:
            similarity_matrix[word2].append(word1)

    all_similarity = defaultdict(dict)
    for each_key in similarity_matrix:
        for each_value in similarity_matrix[each_key]:
            all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])

    final_antonym_list = []
    for index_counter, each_key in enumerate(all_similarity):
        listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
        final_antonym_list.append((each_key, listofTuples[0][0]))

    print(len(final_antonym_list))
    return final_antonym_list

In [170]:
final_antonym_list = collect_antonyms(root, processed_antonyms_wordnet, processed_antonyms_api, model_norm)
#store final antonyms
with open('./generated_antonyms/country_related_antonyms', 'w') as f:
    f.writelines([i+' ' +j+'\n' for i,j in final_antonym_list])

1141


In [171]:
final_antonym_list[:30]

[('close', 'expand'),
 ('help', 'partition'),
 ('fighting', 'nice'),
 ('executive', 'subordinate'),
 ('consumer', 'seller'),
 ('bountiful', 'insufficient'),
 ('commitment', 'detachment'),
 ('awful', 'beautiful'),
 ('current', 'old'),
 ('ambiguous', 'direct'),
 ('actuality', 'forfeit'),
 ('chore', 'peace'),
 ('erudition', 'ignorance'),
 ('good', 'nefarious'),
 ('structured', 'unstructured'),
 ('uncultured', 'urbane'),
 ('gloss', 'misinformation'),
 ('endangerment', 'safeguard'),
 ('opponent', 'patron'),
 ('huge', 'microscopic'),
 ('expert', 'neophyte'),
 ('distribute', 'gather'),
 ('lose', 'profit'),
 ('avoidance', 'encounter'),
 ('agitation', 'silence'),
 ('encumbrance', 'help'),
 ('extreme', 'space'),
 ('float', 'sink'),
 ('indefinite', 'secure'),
 ('antagonistic', 'kind')]